In [1]:
import os
import shutil
import gc
import time
import random as rn
import numpy as np
import pandas as pd
import warnings
import csv

import scipy.io as sio
from scipy import signal
from tqdm import tqdm
import matplotlib.pyplot as plt

from scipy import sparse
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold

# import wfdb
# import wfdb.processing as wp
# from utils import extract_basic_features
# from utils import find_noise_features, extract_basic_features
# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier

''' '''
# from resnet_ecg.utils import one_hot,get_batches
from resnet_ecg.ecg_preprocess import ecg_preprocessing
from resnet_ecg.densemodel import Net
from resnet_ecg import attentionmodel
from keras.utils import to_categorical
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
import keras.backend as K
from keras.layers import Input
from keras.models import Model, load_model
import keras
import pywt


warnings.filterwarnings("ignore")

''' '''
config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
config.gpu_options.per_process_gpu_memory_fraction = 0.8
session = tf.Session(config=config)
KTF.set_session(session)

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
tf.set_random_seed(1234)


# path of training data
path = '/media/jdcloud/'


def wavelet(ecg, wavefunc, lv, m, n):  #

    coeff = pywt.wavedec(ecg, wavefunc, mode='sym', level=lv)  #
    # sgn = lambda x: 1 if x > 0 else -1 if x < 0 else 0

    for i in range(m, n + 1):
        cD = coeff[i]
        for j in range(len(cD)):
            Tr = np.sqrt(2 * np.log(len(cD)))
            if cD[j] >= Tr:
                coeff[i][j] = np.sign(cD[j]) - Tr
            else:
                coeff[i][j] = 0

    denoised_ecg = pywt.waverec(coeff, wavefunc)
    return denoised_ecg


def wavelet_db6(sig):
    """
    R J, Acharya U R, Min L C. ECG beat classification using PCA, LDA, ICA and discrete
     wavelet transform[J].Biomedical Signal Processing and Control, 2013, 8(5): 437-448.
    param sig: 1-D numpy Array
    return: 1-D numpy Array
    """
    coeffs = pywt.wavedec(sig, 'db6', level=9)
    coeffs[-1] = np.zeros(len(coeffs[-1]))
    coeffs[-2] = np.zeros(len(coeffs[-2]))
    coeffs[0] = np.zeros(len(coeffs[0]))
    sig_filt = pywt.waverec(coeffs, 'db6')
    return sig_filt


def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    # Calculates the F score, the weighted harmonic mean of precision and recall.
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    # Calculates the f-measure, the harmonic mean of precision and recall.
    return fbeta_score(y_true, y_pred, beta=1)


def read_data_seg(data_path, split="Train", preprocess=False, fs=500, newFs=256, winSecond=10, winNum=10, n_index=0,pre_type="sym"):
    """ Read data """

    # Fixed params
    # n_index = 0
    n_class = 9
    winSize = winSecond * fs
    new_winSize = winSecond * newFs
    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels and one-hot encode
    # label_path = os.path.join(data_path, "reference.txt")
    # labels = pd.read_csv(label_path, sep='\t',header = None)
    # labels = pd.read_csv("reference.csv")

    # Read time-series data
    channel_files = os.listdir(path_signals)
    # print(channel_files)
    channel_files.sort()
    n_channels = 12  # len(channel_files)
    # posix = len(split) + 5

    # Initiate array
    list_of_channels = []

    X = np.zeros((len(channel_files), new_winSize, n_channels)).astype('float32')
    i_ch = 0

    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1', 'II', 'aVR', 'V3', 'III', 'V5']


    for i_ch, fil_ch in enumerate(channel_files[:]):  # tqdm

        if i_ch % 1000 == 0:
            print(i_ch)

        ecg = sio.loadmat(os.path.join(path_signals, fil_ch))
        ecg_length = ecg["I"].shape[1]

        if ecg_length > fs * winNum * winSecond:
            print(" too long !!!", ecg_length)
            ecg_length = fs * winNum * winSecond
        if ecg_length < 4500:
            print(" too short !!!", ecg_length)
            break

        slide_steps = int((ecg_length - winSize) / winSecond)

        if ecg_length <= 4500:
            slide_steps = 0

        ecg_channels = np.zeros((new_winSize, n_channels)).astype('float32')

        for i_n, ch_name in enumerate(channel_name):

            ecg_channels[:, i_n] = signal.resample(ecg[ch_name]
                                                   [:, n_index * slide_steps:n_index * slide_steps + winSize].T
                                                   , new_winSize).T
            if preprocess:
                if pre_type == "sym":
                    ecg_channels[:, i_n] = ecg_preprocessing(ecg_channels[:, i_n].reshape(1, new_winSize), 'sym8', 8, 3,
                                                             newFs, removebaseline=False, normalize=False)[0]
                elif pre_type == "db4":
                    ecg_channels[:, i_n] = wavelet(ecg_channels[:, i_n], 'db4', 4, 2, 4)
                elif pre_type == "db6":
                    ecg_channels[:, i_n] = wavelet_db6(ecg_channels[:, i_n])

                # ecg_channels[:, i_n] = (ecg_channels[:, i_n]-np.mean(ecg_channels[:, i_n]))/np.std(ecg_channels[:, i_n])
            else:
                pass
                print(" no preprocess !!! ")

        X[i_ch, :, :] = ecg_channels

    return X


def preprocess_y(labels, y, num_class=9):
    bin_label = np.zeros((len(y), num_class)).astype('int8')
    for i in range(len(y)):
        label_nona = labels.loc[y[i]].dropna()
        for j in range(1, label_nona.shape[0]):
            bin_label[i, int(label_nona[j])] = 1
    return bin_label

Using TensorFlow backend.


In [2]:
def read_data(data_path, split="Train", preprocess=True, fs=500, newFs=256, winSecond=10, winNum=10, n_index=0,pre_type="sym"):
    """ Read data """

    # Fixed params
    # n_index = 0
    n_class = 9
    winSize = winSecond * fs
    new_winSize = 23296# winSecond * newFs
    # Paths
    path_signals = os.path.join(data_path, split)

    # Read time-series data
    channel_files = os.listdir(path_signals)
    channel_files.sort()
    n_channels = 12  # len(channel_files)

    X = np.zeros((len(channel_files), new_winSize, n_channels)).astype('float32')

    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1', 'II', 'aVR', 'V3', 'III', 'V5']

    for i_ch, fil_ch in enumerate(channel_files[:]):  # tqdm

        if i_ch % 1000 == 0:
            print(i_ch)

        ecg = sio.loadmat(os.path.join(path_signals, fil_ch))
        ecg_length = ecg["I"].shape[1]
        
        if ecg_length > 45500:
            ecg_length = 45500

        ecg_channels = np.zeros((new_winSize, n_channels)).astype('float32')

        for i_n, ch_name in enumerate(channel_name):

            ecg_data = signal.resample(ecg[ch_name][:,:ecg_length].T, 
                                       int(ecg[ch_name][:,:ecg_length].shape[1] / 500 * newFs)).T
            if preprocess:
                if pre_type == "sym":
                    ecg_channels[-ecg_data.shape[1]:, i_n] = ecg_preprocessing(ecg_data, 'sym8', 8, 3,
                                                    newFs, removebaseline=False, normalize=False)[0]

                elif pre_type == "db4":
                    ecg_channels[-ecg_data.shape[1]:, i_n] = wavelet(ecg_data[0], 'db4', 4, 2, 4)
                elif pre_type == "db6":
                    ecg_channels[-ecg_data.shape[1]:, i_n] = wavelet_db6(ecg_data[0])

                # ecg_channels[:, i_n] = (ecg_channels[:, i_n]-np.mean(ecg_channels[:, i_n]))/np.std(ecg_channels[:, i_n])
            else:
                pass
                print(" no preprocess !!! ")

        X[i_ch, :, :] = ecg_channels

    return X

In [3]:
pre_type = "sym"# "sym"

labels = pd.read_csv(path + "reference.csv")
raw_IDs = labels["File_name"].values.tolist()

IDs = {}
IDs["sym"] = raw_IDs
IDs["db4"] = [i + "_db4" for i in raw_IDs]
IDs["db6"] = [i + "_db6" for i in raw_IDs]

input_size = (2560, 12)
net_num = 10
inputs_list = [Input(shape=input_size) for _ in range(net_num)]
net = Net()
outputs = net.nnet(inputs_list, 0.5, num_classes=9)
model = Model(inputs=inputs_list, outputs=outputs)

net_num = 10
test_x = [read_data_seg(path, split='Val', preprocess=True, n_index=i, pre_type=pre_type) for i in range(net_num)]

model_path = './official_densenet_model/'

en_amount = 1
for seed in range(en_amount):
    print("************************")
    n_fold = 3  # 3
    n_classes = 9

    kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
    kf = kfold.split(IDs[pre_type], labels['label1'])

    densenet_blend_train = np.zeros((6500, n_fold, n_classes)).astype('float32')  # len(train_x)
    densenet_blend_test = np.zeros((500, n_fold, n_classes)).astype('float32')  # len(test_x)

    count = 0

    for i, (index_train, index_valid) in enumerate(kf):
        print('fold: ', i + 1, ' training')
        t = time.time()

        tr_IDs = np.array(IDs[pre_type]) # [index_train]
        # val_IDs = np.array(IDs[pre_type])[index_valid]
        print(tr_IDs.shape)

        X = np.empty((tr_IDs.shape[0], 10, 2560, 12))
        for j, ID in enumerate(tr_IDs):
            X[j, ] = np.load("training_data/" + ID + ".npy")
        # X_tr = [(X[:, i] - np.mean(X[:, i])) / np.std(X[:, i]) for i in range(10)]
        X_tr = [X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5], X[:, 6], X[:, 7], X[:, 8], X[:, 9]]
        # print(X.shape)
        del X

        # Evaluate best trained model
        model.load_weights(model_path + 'densenet_extend_weights-best_k{}_r{}.hdf5'.format(seed, i))

        densenet_blend_train[:, i, :] = model.predict(X_tr)
        densenet_blend_test[:, i, :] = model.predict(test_x)

        del X_tr
        gc.collect()
        gc.collect()
        count += 1

index = np.arange(6500)
y_train = preprocess_y(labels, index)

train_y = 0.1 * densenet_blend_train[:, 0, :] + 0.1 * densenet_blend_train[:, 1, :] + 0.8 * densenet_blend_train[:, 2, :]

threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(train_y.shape[1])

for i in range(train_y.shape[1]):
    y_prob = np.array(train_y[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if train_y[i, j] >= best_threshold[j] else 0 for j in range(train_y.shape[1])]
          for i in range(len(train_y))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(9):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
0
0
0
0
0
0
0
0
0
0
************************
fold:  1  training
(6500,)
fold:  2  training
(6500,)
fold:  3  training
(6500,)
best_threshold : [0.5 0.7 0.6 0.2 0.4 0.2 0.5 0.2 0.4]
 train data f1_score  : 0.9529411674736975
f1 score of ab 0 is 0.9599349129901498
f1 score of ab 1 is 0.9941117520288308
f1 score of ab 2 is 0.9778956060483803
f1 score of ab 3 is 0.99722620747456
f1 score of ab 4 is 0.9767802069020152
f1 score of ab 5 is 0.9915109802205838
f1 score of ab 6 is 0.9845972343293512
f1 score of ab 7 is 0.9148090014119739
f1 score of ab 8 is 0.9560048862533019


In [4]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape, GRU, CuDNNGRU
from keras.layers import Convolution1D, MaxPool1D, GlobalAveragePooling1D,concatenate,AveragePooling1D
from keras.utils import multi_gpu_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from keras.models import Model
from keras.utils import multi_gpu_model
from keras import initializers, regularizers, constraints
from keras.layers import Layer
import numpy as np
from keras.layers.normalization import BatchNormalization
from keras import regularizers

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)
        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
            self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)
        a = K.exp(ait)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [5]:
pre_type = "db6"# "sym"

labels = pd.read_csv(path + "reference.csv")
raw_IDs = labels["File_name"].values.tolist()

IDs = {}
IDs["sym"] = raw_IDs
IDs["db4"] = [i + "_db4" for i in raw_IDs]
IDs["db6"] = [i + "_db6" for i in raw_IDs]

batch_size = 64
num_classes = 9
len_seg = 23296  # 91s

main_input = Input(shape=(len_seg, 12), dtype='float32', name='main_input')
x = Convolution1D(12, 3, padding='same')(main_input)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides=2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides=2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides=2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 24, strides=2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 3, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
x = Convolution1D(12, 48, strides=2, padding='same')(x)
x = LeakyReLU(alpha=0.3)(x)
cnnout = Dropout(0.2)(x)
x = Bidirectional(CuDNNGRU(12, input_shape=(2250, 12), return_sequences=True, return_state=False))(cnnout)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
x = AttentionWithContext()(x)
x = BatchNormalization()(x)
x = LeakyReLU(alpha=0.3)(x)
x = Dropout(0.2)(x)
main_output = Dense(num_classes, activation='sigmoid')(x)
model = Model(inputs=main_input, outputs=main_output)

test_x = read_data(path, split='Val', preprocess=True, n_index=0, pre_type=pre_type)
print("test_x shape: ",test_x.shape)
model_path = '/media/uuser/data/test/official_attention_onenet_model/'


en_amount = 1
for seed in range(en_amount):
    print("************************")
    n_fold = 3  # 3
    n_classes = 9

    kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
    kf = kfold.split(IDs[pre_type], labels['label1'])

    attention_one_blend_train = np.zeros((6500, n_fold, n_classes)).astype('float32')  # len(train_x)
    attention_one_blend_test = np.zeros((500, n_fold, n_classes)).astype('float32')  # len(test_x)

    count = 0

    for i, (index_train, index_valid) in enumerate(kf):
        print('fold: ', i + 1, ' training')
        t = time.time()

        tr_IDs = np.array(IDs[pre_type]) # [index_train]
        # val_IDs = np.array(IDs[pre_type])[index_valid]
        print(tr_IDs.shape)

        X = np.empty((tr_IDs.shape[0], 23296, 12))
        for j, ID in enumerate(tr_IDs):
            X[j, ] = np.load("/media/uuser/data/test/training_data_pre/" + ID + ".npy")
        # X_tr = [(X[:, i] - np.mean(X[:, i])) / np.std(X[:, i]) for i in range(10)]
        X_tr = X
        # print(X.shape)
        del X

        # Evaluate best trained model
        model.load_weights(model_path + 'attention_extend_weights-best_k{}_r{}_0608.hdf5'.format(seed, i))

        attention_one_blend_train[:, i, :] = model.predict(X_tr)
        attention_one_blend_test[:, i, :] = model.predict(test_x)

        del X_tr
        gc.collect()
        gc.collect()
        count += 1

0
test_x shape:  (500, 23296, 12)
************************
fold:  1  training
(6500,)
fold:  2  training
(6500,)
fold:  3  training
(6500,)


In [6]:
index = np.arange(6500)
y_train = preprocess_y(labels, index)

train_y = 0.1 * attention_one_blend_train[:, 0, :] + 0.1 * attention_one_blend_train[:, 1, :] + 0.8 * attention_one_blend_train[:, 2, :]

threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(train_y.shape[1])

for i in range(train_y.shape[1]):
    y_prob = np.array(train_y[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if train_y[i, j] >= best_threshold[j] else 0 for j in range(train_y.shape[1])]
          for i in range(len(train_y))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(9):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))


out = 0.1 * attention_one_blend_test[:, 0, :] + 0.1 * attention_one_blend_test[:, 1, :] + 0.8 * attention_one_blend_test[:, 2, :]

y_pred_test = np.array(
    [[1 if out[i, j] >= best_threshold[j] else 0 for j in range(out.shape[1])] for i in range(len(out))])

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8]

test_y = y_pred_test

y_pred = [[1 if test_y[i, j] >= best_threshold[j] else 0 for j in range(test_y.shape[1])]
          for i in range(len(test_y))]
pred = []
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(9) if y_pred[j][i] == 1])


val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('answers_attention_{}_0607.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(8 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

best_threshold : [0.6 0.5 0.8 0.6 0.5 0.2 0.4 0.4 0.6]
 train data f1_score  : 0.9233701703313789
f1 score of ab 0 is 0.9538712653466751
f1 score of ab 1 is 0.9861697600576124
f1 score of ab 2 is 0.951993192129309
f1 score of ab 3 is 0.9955042073968186
f1 score of ab 4 is 0.9138753269425088
f1 score of ab 5 is 0.9838926937780672
f1 score of ab 6 is 0.9729410873353466
f1 score of ab 7 is 0.9092356407791482
f1 score of ab 8 is 0.9371545803159034


# ensemble densenet and attention one net

In [29]:
index = np.arange(6500)
y_train = preprocess_y(labels, index)

thr = np.array([0.7, 0.7, 0.7, 0.7, 0., 0.7, 0.8, 1., 0.7])

train_y1 =   thr * (0.1 * densenet_blend_train[:, 0, :] + 
            0.1 * densenet_blend_train[:, 1, :] + 
            0.8 * densenet_blend_train[:, 2, :])

train_y2 =  (1-thr) * (0.1 * attention_one_blend_train[:, 0, :] + 
            0.1 * attention_one_blend_train[:, 1, :] + 
            0.8 * attention_one_blend_train[:, 2, :])

train_y = train_y1+train_y2

threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(train_y.shape[1])

for i in range(train_y.shape[1]):
    y_prob = np.array(train_y[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if train_y[i, j] >= best_threshold[j] else 0 for j in range(train_y.shape[1])]
          for i in range(len(train_y))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(9):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))
    
out1 = thr *(0.1 * densenet_blend_test[:, 0, :] + 
        0.1 * densenet_blend_test[:, 1, :] + 
        0.8 * densenet_blend_test[:, 2, :])

out2 = (1-thr) *(0.1 * attention_one_blend_test[:, 0, :] + 
        0.1 * attention_one_blend_test[:, 1, :] + 
        0.8 * attention_one_blend_test[:, 2, :])

out = out1+out2

print("out shape: ",out.shape)
y_pred_test = np.array(
    [[1 if out[i, j] >= best_threshold[j] else 0 for j in range(out.shape[1])] for i in range(len(out))])

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8]

test_y = y_pred_test

y_pred = [[1 if test_y[i, j] >= best_threshold[j] else 0 for j in range(test_y.shape[1])]
          for i in range(len(test_y))]
pred = []
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(9) if y_pred[j][i] == 1])
    
    
    
for i,val in enumerate(pred):
    if 0 in val and len(val) > 1:
        flag = 0
        for j in val:
            if (test_y[i][0] - best_threshold[0]) > (test_y[i][j] - best_threshold[j]):
                pass
            else:
                flag = 1
        if flag == 1:
            pred[i] = val[1:]
        else:
            pred[i] = val[0]
    if len(val) == 0:
        pred[i] = [np.argmin(best_threshold-out[i])]
            
            

val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('answers_attention_{}_ensemble_0610.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(8 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

best_threshold : [0.6 0.6 0.7 0.4 0.5 0.2 0.4 0.2 0.4]
 train data f1_score  : 0.9388255901947657
f1 score of ab 0 is 0.9639216580042775
f1 score of ab 1 is 0.9930662177842734
f1 score of ab 2 is 0.9749852955163361
f1 score of ab 3 is 0.9968778684583539
f1 score of ab 4 is 0.9138753269425088
f1 score of ab 5 is 0.9915453445285112
f1 score of ab 6 is 0.9851604610758894
f1 score of ab 7 is 0.9148090014119739
f1 score of ab 8 is 0.9531725095085233
out shape:  (500, 9)


In [28]:
np.argmin([2,3,1,1,2,2,2,2])

2

In [ ]:
train data f1_score  : 0.9529411674736975
f1 score of ab 0 is 0.9599349129901498
f1 score of ab 1 is 0.9941117520288308
f1 score of ab 2 is 0.9778956060483803
f1 score of ab 3 is 0.99722620747456
f1 score of ab 4 is 0.9767802069020152
f1 score of ab 5 is 0.9915109802205838
f1 score of ab 6 is 0.9845972343293512
f1 score of ab 7 is 0.9148090014119739
f1 score of ab 8 is 0.9560048862533019

In [ ]:
best_threshold : [0.6 0.5 0.8 0.6 0.5 0.2 0.4 0.4 0.6]
train data f1_score  : 0.9233701703313789
f1 score of ab 0 is 0.9538712653466751
f1 score of ab 1 is 0.9861697600576124
f1 score of ab 2 is 0.951993192129309
f1 score of ab 3 is 0.9955042073968186
f1 score of ab 4 is 0.9138753269425088
f1 score of ab 5 is 0.9838926937780672
f1 score of ab 6 is 0.9729410873353466
f1 score of ab 7 is 0.9092356407791482
f1 score of ab 8 is 0.9371545803159034

In [9]:
model_json = model.to_json()  
with open("attention_one_net_model.json", "w") as json_file:  
    json_file.write(model_json)  

In [19]:
pre_type = "sym"# "sym"

labels = pd.read_csv(path + "reference.csv")
raw_IDs = labels["File_name"].values.tolist()

IDs = {}
IDs["sym"] = raw_IDs
IDs["db4"] = [i + "_db4" for i in raw_IDs]
IDs["db6"] = [i + "_db6" for i in raw_IDs]

input_size = (2560, 12)
net_num = 10
inputs_list = [Input(shape=input_size) for _ in range(net_num)]
outputs = attentionmodel.build_network(inputs_list, 0.5, num_classes=9, block_size=4, relu=False)
model = Model(inputs=inputs_list, outputs=outputs)

net_num = 10
test_x = [read_data_seg(path, split='Val', preprocess=True, n_index=i, pre_type=pre_type) for i in range(net_num)]

model_path = './official_attention_model/'

en_amount = 1
for seed in range(en_amount):
    print("************************")
    n_fold = 3  # 3
    n_classes = 9

    kfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
    kf = kfold.split(IDs[pre_type], labels['label1'])

    attention_blend_train = np.zeros((6500, n_fold, n_classes)).astype('float32')  # len(train_x)
    attention_blend_test = np.zeros((500, n_fold, n_classes)).astype('float32')  # len(test_x)

    count = 0

    for i, (index_train, index_valid) in enumerate(kf):
        print('fold: ', i + 1, ' training')
        t = time.time()

        tr_IDs = np.array(IDs[pre_type]) # [index_train]
        # val_IDs = np.array(IDs[pre_type])[index_valid]
        print(tr_IDs.shape)

        X = np.empty((tr_IDs.shape[0], 10, 2560, 12))
        for j, ID in enumerate(tr_IDs):
            X[j, ] = np.load("training_data/" + ID + ".npy")
        # X_tr = [(X[:, i] - np.mean(X[:, i])) / np.std(X[:, i]) for i in range(10)]
        X_tr = [X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5], X[:, 6], X[:, 7], X[:, 8], X[:, 9]]
        # print(X.shape)
        del X

        # Evaluate best trained model
        model.load_weights(model_path + 'attention_extend_weights-best_k{}_r{}_0608.hdf5'.format(seed, i))

        attention_blend_train[:, i, :] = model.predict(X_tr)
        attention_blend_test[:, i, :] = model.predict(test_x)

        del X_tr
        gc.collect()
        gc.collect()
        count += 1

index = np.arange(6500)
y_train = preprocess_y(labels, index)

train_y = 0.1 * attention_blend_train[:, 0, :] + 0.1 * attention_blend_train[:, 1, :] + 0.8 * attention_blend_train[:, 2, :]

threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(train_y.shape[1])

for i in range(train_y.shape[1]):
    y_prob = np.array(train_y[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if train_y[i, j] >= best_threshold[j] else 0 for j in range(train_y.shape[1])]
          for i in range(len(train_y))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(9):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))

0
0
0
0
0
0
0
0
0
0
************************
fold:  1  training
(6500,)
fold:  2  training
(6500,)
fold:  3  training
(6500,)
best_threshold : [0.6 0.8 0.5 0.2 0.5 0.2 0.2 0.2 0.5]
 train data f1_score  : 0.9620003898169097
f1 score of ab 0 is 0.9732364328811891
f1 score of ab 1 is 0.998388285763381
f1 score of ab 2 is 0.9792111549900056
f1 score of ab 3 is 0.9965363414515095
f1 score of ab 4 is 0.9843293819267146
f1 score of ab 5 is 0.9854501844600596
f1 score of ab 6 is 0.9746731964042199
f1 score of ab 7 is 0.9429924064453483
f1 score of ab 8 is 0.9659495598771206


In [22]:
import xgboost as xgb
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer

In [21]:
OneVsRestClassifier(LogisticRegression(C=100))

OneVsRestClassifier(estimator=LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None)

In [22]:
attention_blend_train.shape

(6500, 3, 9)

In [23]:
y_train.shape

(6500, 9)

In [40]:
#X_train = np.hstack((attention_blend_train[:, 0, :] , attention_blend_train[:, 1, :], attention_blend_train[:, 2, :]))
#X_train = np.hstack((attention_one_blend_train[:, 0, :] , 
#                     attention_one_blend_train[:, 1, :], 
#                     attention_one_blend_train[:, 2, :]))
x1 =  (0.1 * densenet_blend_train[:, 0, :] + 
            0.1 * densenet_blend_train[:, 1, :] + 
            0.8 * densenet_blend_train[:, 2, :]) 
x2 =  (0.1 * attention_one_blend_train[:, 0, :] + 
            0.1 * attention_one_blend_train[:, 1, :] + 
            0.8 * attention_one_blend_train[:, 2, :])

X_train = np.hstack((x1,x2 ))

In [45]:
X_train.shape

(6500, 18)

In [42]:
#clf = OneVsRestClassifier(xgb.XGBClassifier(learning_rate=0.005,n_eatimators=500))
clf = OneVsRestClassifier(LogisticRegression(C=100))
clf.fit(X_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None)

In [46]:
#X_test = np.hstack((attention_blend_test[:, 0, :] , attention_blend_test[:, 1, :], attention_blend_test[:, 2, :]))
#X_test = np.hstack((attention_one_blend_test[:, 0, :] , 
#                    attention_one_blend_test[:, 1, :], 
#                    attention_one_blend_test[:, 2, :]))
xt1 = (0.1 * densenet_blend_test[:, 0, :] + 
        0.1 * densenet_blend_test[:, 1, :] + 
        0.8 * densenet_blend_test[:, 2, :])

xt2 = (0.1 * attention_one_blend_test[:, 0, :] + 
        0.1 * attention_one_blend_test[:, 1, :] + 
        0.8 * attention_one_blend_test[:, 2, :])

X_test = np.hstack((xt1,xt2))

In [47]:
y_p_train = clf.predict_proba(X_train)
y_p_test = clf.predict_proba(X_test)

In [49]:
threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(y_p_train.shape[1])

for i in range(y_p_train.shape[1]):
    y_prob = np.array(y_p_train[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if y_p_train[i, j] >= best_threshold[j] else 0 for j in range(y_p_train.shape[1])]
          for i in range(len(y_p_train))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(9):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))

best_threshold : [0.3 0.7 0.5 0.4 0.3 0.3 0.2 0.2 0.4]
 train data f1_score  : 0.9589222099588254
f1 score of ab 0 is 0.9704663844317845
f1 score of ab 1 is 0.9962461468347303
f1 score of ab 2 is 0.9792812723926492
f1 score of ab 3 is 0.9975716754676085
f1 score of ab 4 is 0.9831316484012695
f1 score of ab 5 is 0.9910562034074882
f1 score of ab 6 is 0.987649835989822
f1 score of ab 7 is 0.9207155546754144
f1 score of ab 8 is 0.958464732990536


In [50]:
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8]

test_y = y_p_test

y_pred = [[1 if test_y[i, j] >= best_threshold[j] else 0 for j in range(test_y.shape[1])]
          for i in range(len(test_y))]
pred = []
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(9) if y_pred[j][i] == 1])

In [52]:
pre_type

'sym'

In [51]:
val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('answers_densenet_{}_ensemble_0609_3.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(8 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

In [33]:
train_y = 0. * attention_blend_train[:, 0, :] + 0.2 * attention_blend_train[:, 1, :] + 0.8 * attention_blend_train[:, 2, :]

threshold = np.arange(0.1, 0.9, 0.1)
acc = []
accuracies = []
best_threshold = np.zeros(train_y.shape[1])

for i in range(train_y.shape[1]):
    y_prob = np.array(train_y[:, i])
    for j in threshold:
        y_pred = [1 if prob >= j else 0 for prob in y_prob]
        acc.append(f1_score(y_train[:, i], y_pred, average='macro'))
    acc = np.array(acc)
    index = np.where(acc == acc.max())
    accuracies.append(acc.max())
    best_threshold[i] = threshold[index[0][0]]
    acc = []

print("best_threshold :", best_threshold)

y_pred = np.array([[1 if train_y[i, j] >= best_threshold[j] else 0 for j in range(train_y.shape[1])]
          for i in range(len(train_y))])
print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(9):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))

out = 0. * attention_blend_test[:, 0, :] + 0.2 * attention_blend_test[:, 1, :] + 0.8 * attention_blend_test[:, 2, :]

y_pred_test = np.array(
    [[1 if out[i, j] >= best_threshold[j] else 0 for j in range(out.shape[1])] for i in range(len(out))])

classes = [0, 1, 2, 3, 4, 5, 6, 7, 8]

test_y = y_pred_test

y_pred = [[1 if test_y[i, j] >= best_threshold[j] else 0 for j in range(test_y.shape[1])]
          for i in range(len(test_y))]
pred = []
for j in range(test_y.shape[0]):
    pred.append([classes[i] for i in range(9) if y_pred[j][i] == 1])
    
val_dataset_path = path + "/Val/"
val_files = os.listdir(val_dataset_path)
val_files.sort()

with open('answers_densenet_{}_ensemble_0609_2.csv'.format(pre_type), 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File_name', 'label1', 'label2',
                     'label3', 'label4', 'label5', 'label6', 'label7', 'label8'])
    count = 0
    for file_name in val_files:
        if file_name.endswith('.mat'):

            record_name = file_name.strip('.mat')
            answer = []
            answer.append(record_name)

            result = pred[count]

            answer.extend(result)
            for i in range(8 - len(result)):
                answer.append('')
            count += 1
            writer.writerow(answer)
    csvfile.close()

best_threshold : [0.6 0.8 0.5 0.2 0.5 0.1 0.2 0.2 0.5]
 train data f1_score  : 0.9628445837768207
f1 score of ab 0 is 0.9735929889203465
f1 score of ab 1 is 0.998388285763381
f1 score of ab 2 is 0.9787580310982424
f1 score of ab 3 is 0.9968810963570086
f1 score of ab 4 is 0.9843293819267146
f1 score of ab 5 is 0.9864175683529343
f1 score of ab 6 is 0.9759978037672116
f1 score of ab 7 is 0.9445165255394701
f1 score of ab 8 is 0.9661107445538584
